In [1]:
import json
import os
import numpy as np
import pandas as pd

identifier = []
photos = []
captions = []
collections = 0
photo_dump = 0
num_dump = 0
no_image = 0
no_caption = 0
has_quotes = 0
scraper_failed = 0
for fn in os.listdir("profiles_sejal/"):
    if fn.endswith(".json"):
        f = open("profiles_sejal/" + fn)
        num_valid_entries = 0
        num_invalid_entries = 0
        data = json.load(f)
        try:
            posts = data["posts"]
        except:
            if data["num_of_posts"] > 0:
                scraper_failed += data["num_of_posts"]
            exit
        for p in posts:
            if len(p["imgs"]) == 0:
                no_image += 1
                num_valid_entries += 1
            elif p["caption"] == "":
                no_caption += 1
                num_invalid_entries += 1
            else:
                num_valid_entries += 1
                if len(p["imgs"]) > 1:
                    print("collection")
                    collections += 1
                identifier.append(data["username"])
                photos.append(p["imgs"][0])
                if '"' in p["caption"]:
                    has_quotes += 1
                captions.append(p["caption"].replace('"', '\''))
        scraper_failed += data["num_of_posts"] - num_valid_entries - num_invalid_entries
        f.close()

total = len(captions) + photo_dump + collections + no_image + no_caption + scraper_failed
print("throwing out:", photo_dump + no_image + no_caption + scraper_failed, round((photo_dump + scraper_failed + no_image + no_caption) * 100 / total, 2))
print("\t no caption", no_caption, round((no_caption / total) * 100, 2))
print("\t no image", no_image, round((no_image / total) * 100, 2))
print("\t scraper failed", scraper_failed, round(scraper_failed * 100/ total, 2))
print("usable:", len(captions) + collections, round((len(captions)+ collections)*100/total, 2))
print("\t valid collections:", collections, round(collections* 100 / total, 2))
print("\t valid single photos:", len(captions), round(len(captions)*100/total, 2))

with open('data_sejal.csv', 'w') as f:
    f.write("username,photo,caption\n")
    for i in range(len(photos)):
        f.write("%s,%s,\"%s\"\n" % (identifier[i],photos[i], captions[i]))
f.close()

try:
    csv = pd.read_csv("data_sejal.csv", header=0)
    csv.drop_duplicates(keep="first",inplace=True)
    csv.to_csv("data_sejal.csv", index=False)
except:
    print('pandas stuff did not work')


throwing out: 82168 74.49
	 no caption 13310 12.07
	 no image 934 0.85
	 scraper failed 67924 61.58
usable: 28134 25.51
	 valid collections: 0 0.0
	 valid single photos: 28134 25.51
